This notebook prepares that data by putting it in catalogue.
It is specific for how the data was stored on our servers.

It calculates delta in percentage for cdca.
It also regrids PC on CDCA.

In [1]:
import xarray as xr
from matplotlib import pyplot as plt
import glob
from xscen.config import CONFIG, load_config
import cartopy.crs as ccrs
import pandas as pd
import xclim as xc
import spirograph.matplotlib as sp
import string
from dask.distributed import Client


import xscen as xs
from dask.diagnostics import ProgressBar

load_config('paths_pc.yml', verbose=(__name__ == '__main__'), reset=True)
tdd={'xarray_open_kwargs':{'decode_timedelta': False}}
pd.set_option('display.max_columns', None)
sp.utils.set_mpl_style('ouranos')

cbar= {'tg_mean': 'plasma',
       'tx_mean': 'plasma',
       'tn_mean': 'plasma',
       'prcptot': 'viridis_r'
      }

cbar_bias= {'tg_mean': 'temp_div',
       'tx_mean': 'temp_div',
       'tn_mean': 'temp_div',
       'prcptot': 'prec_div'
      }
time_name= {'AS-JAN':{'1': 'annual'},
           'QS-DEC':{'3':'MAM',
                     '6':'JJA',
                     '9':'SON',
                     '12':'DEC'}}

var2name={'prcptot':r'$\tt{pr}$', 'tx_mean':r'$\tt{tasmax}$', 'tn_mean':r'$\tt{tasmin}$'}


map_proj = ccrs.LambertConformal(central_longitude=-95, central_latitude=45)


INFO:xscen.config:Updated the config with paths_pc.yml.


# Initialize project

In [2]:
project={'name': 'platform-comparison',
 'version': '1.0.0',
 'description': "Comparing canadian climate platform",
 'id': 'pc'}
#pcat = xs.ProjectCatalog.create(CONFIG['paths']['project_catalog'], project=project, overwrite=True)
pcat_espo = xs.ProjectCatalog(CONFIG['paths']['pcat_espo'])
pcat = xs.ProjectCatalog(CONFIG['paths']['project_catalog'])
pcat.df


,id,type,processing_level,bias_adjust_institution,bias_adjust_project,mip_era,activity,driving_institution,driving_model,institution,source,experiment,member,xrfreq,frequency,variable,domain,date_start,date_end,version,format,path
0,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,NaN,NaN,NaN,NaN,ssp126,NaN,MS,mon,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,NaN,nc,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp12...
1,climatedata_CMIP6_ScenarioMIP_ssp585_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,NaN,NaN,NaN,NaN,ssp585,NaN,MS,mon,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,NaN,nc,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp58...
2,climatedata_CMIP6_ScenarioMIP_ssp245_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,NaN,NaN,NaN,NaN,ssp245,NaN,MS,mon,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,NaN,nc,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp24...
3,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,NaN,NaN,NaN,NaN,ssp126,NaN,QS-DEC,qtr,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,NaN,nc,/tank/jlavoie/ccdp/output/final/r20mm/QS-DEC/s...
4,climatedata_CMIP6_ScenarioMIP_ssp585_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,NaN,NaN,NaN,NaN,ssp585,NaN,QS-DEC,qtr,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,NaN,nc,/tank/jlavoie/ccdp/output/final/r20mm/QS-DEC/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,climatedata_CMIP6_ScenarioMIP_KACE-1-0-G_ssp24...,simulation,abs-delta,PCIC,climatedata,CMIP6,ScenarioMIP,NaN,NaN,NaN,KACE-1-0-G,ssp245,NaN,AS-JUL,yr,"(fdd_380_273_delta_1991_2020,)",CAN-PCIC,1951-07-01 00:00,2071-07-01 00:00,NaN,zarr,/jarre/scenario/jlavoie/plat-comp/indicators/C...
6123,climatedata_CMIP6_ScenarioMIP_NorESM2-LM_ssp58...,simulation,abs-delta,PCIC,climatedata,CMIP6,ScenarioMIP,NaN,NaN,NaN,NorESM2-LM,ssp585,NaN,AS-JUL,yr,"(fdd_380_delta_1991_2020, fdd_380_273_delta_19...",CAN-PCIC,1951-07-01 00:00,2071-07-01 00:00,NaN,zarr,/jarre/scenario/jlavoie/plat-comp/indicators/C...
6124,climatedata_CMIP6_ScenarioMIP_MPI-ESM1-2-LR_ss...,simulation,abs-delta,PCIC,climatedata,CMIP6,ScenarioMIP,NaN,NaN,NaN,MPI-ESM1-2-LR,ssp585,NaN,AS-JUL,yr,"(fdd_380_delta_1991_2020, fdd_380_273_delta_19...",CAN-PCIC,1951-07-01 00:00,2071-07-01 00:00,NaN,zarr,/jarre/scenario/jlavoie/plat-comp/indicators/C...
6125,climatedata_CMIP6_ScenarioMIP_ssp585_CAN-PCIC,simulation,ensemble-abs-delta,PCIC,climatedata,CMIP6,ScenarioMIP,NaN,NaN,NaN,NaN,ssp585,NaN,AS-JUL,yr,"(fdd_380_273_delta_1991_2020_p50, fdd_380_273_...",CAN-PCIC,1951-07-01 00:00,2071-07-01 00:00,NaN,zarr,/jarre/scenario/jlavoie/plat-comp/indicators/C...


# add pcic to pcat

In [97]:
# put pcic data in catalog (30ymean Ensemble)
data_pcic_nc_ens30 = xs.catalog.parse_directory(
            directories=[
                        f"{CONFIG['paths']['pcic_data']}/final/",
                          f"{CONFIG['paths']['pcic_data']}/finalPCIC/",
                          f"{CONFIG['paths']['pcic_data']}/finalPCIC_corrige_QS-DEC/",
                        ],
            globpattern='*.*',
            patterns=[
                "{*}/{xrfreq}/{experiment}/ensemble_percentiles/"\
                "{*}_{*}_BCCAQ2v2+ANUSPLIN300_historical+{experiment}_{date_start}-{date_end}_30ymean_percentiles.{*}",
            ],
            read_from_file= ['variable'],
            homogenous_info={
                'type':'simulation',
                'bias_adjust_project':'climatedata',
                'processing_level':'raw',
                'mip_era': 'CMIP6',
                'bias_adjust_institution':'PCIC',
                'activity': 'ScenarioMIP',
                'domain':'CAN-PCIC',
                'processing_level': 'ensemble-30ymean'
            },
        )
data_pcic_nc_ens30= data_pcic_nc_ens30.replace('YS', 'AS-JAN')
data_pcic_nc_ens30=data_pcic_nc_ens30[data_pcic_nc_ens30['path'].str.contains('finalPCIC/prcptot/QS-DEC/')==False]
data_pcic_nc_ens30=data_pcic_nc_ens30[data_pcic_nc_ens30['path'].str.contains('finalPCIC/tx_mean/QS-DEC/')==False]
data_pcic_nc_ens30=data_pcic_nc_ens30[data_pcic_nc_ens30['path'].str.contains('finalPCIC/tn_mean/QS-DEC/')==False]

display(data_pcic_nc_ens30)
pcat.update(data_pcic_nc_ens30)

2023-01-07 15:48:58,402 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/rx1day/YS/ssp245/ensemble_percentiles/rx1day_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:48:58,828 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tx_max/MS/ssp585/ensemble_percentiles/tx_max_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:49:00,981 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tn_min/MS/ssp245/ensemble_percentiles/tn_min_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:49:01,594 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tn_mean/YS/ssp585/ensemble_percentiles/tn_mean_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:49:02,79

2023-01-07 15:49:29,682 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/prcptot/YS/ssp585/ensemble_percentiles/prcptot_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:49:30,103 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tn_mean/MS/ssp245/ensemble_percentiles/tn_mean_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:49:30,196 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tx_mean/QS-DEC/ssp245/ensemble_percentiles/tx_mean_sea_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:49:30,965 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/prcptot/YS/ssp245/ensemble_percentiles/prcptot_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
[           

[##########                              ] | 25% Completed | 67.73 s2023-01-07 15:50:38,883 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/txgt_30/YS/ssp126/ensemble_percentiles/txgt_30_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:50:38,943 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/hddheat_18/YS/ssp585/ensemble_percentiles/hddheat_18_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
[##########                              ] | 25% Completed | 68.25 s2023-01-07 15:50:39,376 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/ccdcold_18/YS/ssp245/ensemble_percentiles/ccdcold_18_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
[##########                              ] | 25% Completed | 68.71 s2023-01-07 15:50:39,850 - xscen.catalog - INFO

[################                        ] | 41% Completed | 95.06 s2023-01-07 15:51:06,233 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/rx1day/YS/ssp585/ensemble_percentiles/rx1day_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:51:06,258 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tn_mean/QS-DEC/ssp585/ensemble_percentiles/tn_mean_sea_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:51:06,282 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/rx1day/YS/ssp126/ensemble_percentiles/rx1day_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_30ymean_percentiles.nc.
[################                        ] | 41% Completed | 95.48 s2023-01-07 15:51:06,671 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tn_m

[###########################             ] | 68% Completed | 165.40 s2023-01-07 15:52:16,602 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/txgt_32/MS/ssp245/ensemble_percentiles/txgt_32_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
[###########################             ] | 69% Completed | 165.55 s2023-01-07 15:52:16,603 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tg_mean/QS-DEC/ssp585/ensemble_percentiles/tg_mean_sea_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
2023-01-07 15:52:16,956 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tnlt_m15/MS/ssp126/ensemble_percentiles/tnlt_m15_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_30ymean_percentiles.nc.
[###########################             ] | 69% Completed | 167.69 s2023-01-07 15:52:19,096 - xscen.catalog - INFO - Parsing attr

[#############################           ] | 73% Completed | 212.49 s2023-01-07 15:53:03,654 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tnlt_m25/YS/ssp126/ensemble_percentiles/tnlt_m25_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_30ymean_percentiles.nc.
[#############################           ] | 73% Completed | 213.46 s2023-01-07 15:53:04,672 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/gddgrow_0/MS/ssp245/ensemble_percentiles/gddgrow_0_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
[#############################           ] | 73% Completed | 215.26 s2023-01-07 15:53:06,445 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/r10mm/MS/ssp585/ensemble_percentiles/r10mm_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
[#############################           ] | 73% Completed | 220.

[#############################           ] | 74% Completed | 270.71 s2023-01-07 15:54:02,131 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tnlt_m25/MS/ssp585/ensemble_percentiles/tnlt_m25_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
[#############################           ] | 74% Completed | 271.63 s2023-01-07 15:54:03,095 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tr_18/MS/ssp585/ensemble_percentiles/tr_18_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
[#############################           ] | 74% Completed | 278.30 s2023-01-07 15:54:09,553 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/r20mm/QS-DEC/ssp245/ensemble_percentiles/r20mm_sea_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
[#############################           ] | 74% Completed | 279.11 s

[####################################    ] | 92% Completed | 464.89 s2023-01-07 15:57:16,098 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/rx5day/YS/ssp245/ensemble_percentiles/rx5day_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_30ymean_percentiles.nc.
[####################################    ] | 92% Completed | 510.97 s2023-01-07 15:58:02,107 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/first_fall_frost/YS/ssp585/ensemble_percentiles/first_fall_frost_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_30ymean_percentiles.nc.
[####################################    ] | 92% Completed | 520.16 s2023-01-07 15:58:11,422 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/dlyfrzthw_tx0_tn-1/YS/ssp126/ensemble_percentiles/dlyfrzthw_tx0_tn-1_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_30ymean_percentiles.nc.
[############################

,id,type,processing_level,bias_adjust_institution,bias_adjust_project,mip_era,activity,driving_institution,driving_model,institution,source,experiment,member,xrfreq,frequency,variable,domain,date_start,date_end,version,format,path
28601,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp126,None,MS,mon,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp12...
79354,climatedata_CMIP6_ScenarioMIP_ssp585_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp585,None,MS,mon,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp58...
90433,climatedata_CMIP6_ScenarioMIP_ssp245_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp245,None,MS,mon,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp24...
119034,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp126,None,QS-DEC,qtr,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/QS-DEC/s...
158712,climatedata_CMIP6_ScenarioMIP_ssp585_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp585,None,QS-DEC,qtr,"(r20mm_delta_1971_2000_p10, r20mm_delta_1971_2...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/QS-DEC/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10247273,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp126,None,QS-DEC,qtr,"(tn_mean_delta_1971_2000_p10, tn_mean_delta_19...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...
10247361,climatedata_CMIP6_ScenarioMIP_ssp245_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp245,None,QS-DEC,qtr,"(tn_mean_delta_1971_2000_p10, tn_mean_delta_19...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...
10247425,climatedata_CMIP6_ScenarioMIP_ssp245_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp245,None,QS-DEC,qtr,"(prcptot_delta_1971_2000_p10, prcptot_delta_19...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...
10247488,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp126,None,QS-DEC,qtr,"(prcptot_delta_1971_2000_p10, prcptot_delta_19...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...


In [98]:
# put pcic data in catalog (year Ensemble)

data_pcic_nc_ens = xs.catalog.parse_directory(
            directories=[
                        f"{CONFIG['paths']['pcic_data']}/final/",
                        f"{CONFIG['paths']['pcic_data']}/finalPCIC/",
                        f"{CONFIG['paths']['pcic_data']}/finalPCIC_corrige_QS-DEC/",
                         
                        ],
            globpattern='*.*',
            patterns=[
                "{*}/{xrfreq}/{experiment}/ensemble_percentiles/"\
                "{*}_{*}_BCCAQ2v2+ANUSPLIN300_historical+{experiment}_{date_start}-{date_end}_percentiles.{*}",
            ],
            read_from_file= ['variable'],
            homogenous_info={
                'type':'simulation',
                'bias_adjust_project':'climatedata',
                'processing_level':'raw',
                'mip_era': 'CMIP6',
                'bias_adjust_institution':'PCIC',
                'activity': 'ScenarioMIP',
                'domain':'CAN-PCIC',
                'processing_level': 'ensemble-indicators'
            },
        )
data_pcic_nc_ens= data_pcic_nc_ens.replace('YS', 'AS-JAN')
data_pcic_nc_ens=data_pcic_nc_ens[data_pcic_nc_ens['path'].str.contains('finalPCIC/prcptot/QS-DEC/')==False]
data_pcic_nc_ens=data_pcic_nc_ens[data_pcic_nc_ens['path'].str.contains('finalPCIC/tx_mean/QS-DEC/')==False]
data_pcic_nc_ens=data_pcic_nc_ens[data_pcic_nc_ens['path'].str.contains('finalPCIC/tn_mean/QS-DEC/')==False]

display(data_pcic_nc_ens)
pcat.update(data_pcic_nc_ens)

[                                        ] | 2% Completed | 15.84 sus2023-01-07 16:01:29,100 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tn_mean/QS-DEC/ssp585/ensemble_percentiles/tn_mean_sea_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_percentiles.nc.
[##                                      ] | 6% Completed | 20.35 s2023-01-07 16:01:29,104 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tx_mean/QS-DEC/ssp245/ensemble_percentiles/tx_mean_sea_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_percentiles.nc.
2023-01-07 16:01:31,271 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/prcptot/QS-DEC/ssp245/ensemble_percentiles/prcptot_sea_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_percentiles.nc.
2023-01-07 16:01:31,338 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/j

[#########                               ] | 23% Completed | 48.04 s2023-01-07 16:01:56,816 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/txgt_25/YS/ssp126/ensemble_percentiles/txgt_25_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_percentiles.nc.
2023-01-07 16:01:57,088 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tn_mean/QS-DEC/ssp585/ensemble_percentiles/tn_mean_sea_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_percentiles.nc.
[#########                               ] | 24% Completed | 48.89 s2023-01-07 16:01:57,834 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/r20mm/YS/ssp585/ensemble_percentiles/r20mm_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_percentiles.nc.
[##########                              ] | 26% Completed | 49.11 s2023-01-07 16:01:57,855 - xscen.catalog - INFO - Parsing attributes with netCDF4 f

[###################                     ] | 49% Completed | 83.80 s2023-01-07 16:02:32,736 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tn_min/YS/ssp585/ensemble_percentiles/tn_min_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_percentiles.nc.
[###################                     ] | 49% Completed | 84.52 s2023-01-07 16:02:33,373 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tr_20/MS/ssp245/ensemble_percentiles/tr_20_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_percentiles.nc.
2023-01-07 16:02:33,938 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC/tx_mean/MS/ssp245/ensemble_percentiles/tx_mean_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_percentiles.nc.
[####################                    ] | 50% Completed | 85.47 s2023-01-07 16:02:34,260 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/

[############################            ] | 71% Completed | 150.10 s2023-01-07 16:03:39,632 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tr_18/YS/ssp126/ensemble_percentiles/tr_18_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_percentiles.nc.
[############################            ] | 71% Completed | 151.86 s2023-01-07 16:03:40,823 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tr_22/YS/ssp245/ensemble_percentiles/tr_22_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_percentiles.nc.
[############################            ] | 71% Completed | 152.49 s2023-01-07 16:03:41,455 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/r20mm/MS/ssp126/ensemble_percentiles/r20mm_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_percentiles.nc.
[############################            ] | 71% Completed | 154.11 s2023-01-07 16:03:43,072 - xscen.ca

[#############################           ] | 74% Completed | 196.77 s2023-01-07 16:04:25,703 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/r20mm/QS-DEC/ssp126/ensemble_percentiles/r20mm_sea_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_percentiles.nc.
[#############################           ] | 74% Completed | 197.61 s2023-01-07 16:04:26,554 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tr_18/MS/ssp585/ensemble_percentiles/tr_18_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_percentiles.nc.
[#############################           ] | 74% Completed | 197.99 s2023-01-07 16:04:26,972 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/tnlt_m25/MS/ssp245/ensemble_percentiles/tnlt_m25_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp245_1950-2100_percentiles.nc.
[#############################           ] | 74% Completed | 201.93 s2023-01-07 16:04:31,036 

[#####################################   ] | 93% Completed | 476.01 s2023-01-07 16:09:04,934 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/first_fall_frost/YS/ssp585/ensemble_percentiles/first_fall_frost_ann_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_percentiles.nc.
[####################################### ] | 97% Completed | 579.48 s2023-01-07 16:10:48,233 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/nr_cdd/MS/ssp585/ensemble_percentiles/nr_cdd_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp585_1950-2100_percentiles.nc.
[####################################### ] | 98% Completed | 586.11 s2023-01-07 16:10:54,987 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/final/nr_cdd/MS/ssp126/ensemble_percentiles/nr_cdd_mon_BCCAQ2v2+ANUSPLIN300_historical+ssp126_1950-2100_percentiles.nc.
[####################################### ] | 98% Completed | 592.75 s2023-01-

,id,type,processing_level,bias_adjust_institution,bias_adjust_project,mip_era,activity,driving_institution,driving_model,institution,source,experiment,member,xrfreq,frequency,variable,domain,date_start,date_end,version,format,path
28600,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-indicators,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp126,None,MS,mon,"(r20mm_p10, r20mm_p50, r20mm_p90)",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp12...
79355,climatedata_CMIP6_ScenarioMIP_ssp585_CAN-PCIC,simulation,ensemble-indicators,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp585,None,MS,mon,"(r20mm_p10, r20mm_p50, r20mm_p90)",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp58...
90432,climatedata_CMIP6_ScenarioMIP_ssp245_CAN-PCIC,simulation,ensemble-indicators,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp245,None,MS,mon,"(r20mm_p10, r20mm_p50, r20mm_p90)",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp24...
119035,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-indicators,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp126,None,QS-DEC,qtr,"(r20mm_p10, r20mm_p50, r20mm_p90)",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/QS-DEC/s...
158713,climatedata_CMIP6_ScenarioMIP_ssp585_CAN-PCIC,simulation,ensemble-indicators,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp585,None,QS-DEC,qtr,"(r20mm_p10, r20mm_p50, r20mm_p90)",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/final/r20mm/QS-DEC/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10247274,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-indicators,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp126,None,QS-DEC,qtr,"(tn_mean_p10, tn_mean_p50, tn_mean_p90)",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...
10247362,climatedata_CMIP6_ScenarioMIP_ssp245_CAN-PCIC,simulation,ensemble-indicators,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp245,None,QS-DEC,qtr,"(tn_mean_p10, tn_mean_p50, tn_mean_p90)",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...
10247424,climatedata_CMIP6_ScenarioMIP_ssp245_CAN-PCIC,simulation,ensemble-indicators,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp245,None,QS-DEC,qtr,"(prcptot_p10, prcptot_p50, prcptot_p90)",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...
10247487,climatedata_CMIP6_ScenarioMIP_ssp126_CAN-PCIC,simulation,ensemble-indicators,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,None,ssp126,None,QS-DEC,qtr,"(prcptot_p10, prcptot_p50, prcptot_p90)",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...


In [4]:
# put pcic data in catalog (30y avg each sim)

data_pcic_30 = xs.catalog.parse_directory(
            directories=[
                        f"{CONFIG['paths']['pcic_data']}/final/",
                        f"{CONFIG['paths']['pcic_data']}/finalPCIC/",
                        f"{CONFIG['paths']['pcic_data']}/finalPCIC_corrige_QS-DEC/",
                         
                        ],
            globpattern='*.*',
            patterns=[
                "{*}/{xrfreq}/{experiment}/simulations_30yAvg/"\
                "{*}_{*}_BCCAQ2v2+ANUSPLIN300_{source}_historical+{experiment}_{date_start}-{date_end}_30ymean.{*}",
            ],
            read_from_file= ['variable'],
            homogenous_info={
                'type':'simulation',
                'bias_adjust_project':'climatedata',
                'processing_level':'raw',
                'mip_era': 'CMIP6',
                'bias_adjust_institution':'PCIC',
                'activity': 'ScenarioMIP',
                'domain':'CAN-PCIC',
                'processing_level': '30ymean'
            },
        )
data_pcic_30=data_pcic_30[data_pcic_30['path'].str.contains('finalPCIC/prcptot/QS-DEC/')==False]
data_pcic_30=data_pcic_30[data_pcic_30['path'].str.contains('finalPCIC/tx_mean/QS-DEC/')==False]
data_pcic_30=data_pcic_30[data_pcic_30['path'].str.contains('finalPCIC/tn_mean/QS-DEC/')==False]


data_pcic_30= data_pcic_30.replace('YS', 'AS-JAN')

display(data_pcic_30)
#pcat.update(data_pcic_30)

[#                                       ] | 3% Completed | 5.72 s ms2023-02-03 13:05:45,814 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/prcptot/QS-DEC/ssp245/simulations_30yAvg/prcptot_sea_BCCAQ2v2+ANUSPLIN300_INM-CM5-0_historical+ssp245_1950-2100_30ymean.nc.
[#                                       ] | 4% Completed | 6.62 s2023-02-03 13:05:46,736 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/prcptot/QS-DEC/ssp245/simulations_30yAvg/prcptot_sea_BCCAQ2v2+ANUSPLIN300_MIROC-ES2L_historical+ssp245_1950-2100_30ymean.nc.
[#                                       ] | 4% Completed | 7.18 s2023-02-03 13:05:47,269 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/prcptot/QS-DEC/ssp245/simulations_30yAvg/prcptot_sea_BCCAQ2v2+ANUSPLIN300_NorESM2-MM_historical+ssp245_1950-2100_30ymean.nc.
[#            

[##                                      ] | 6% Completed | 30.50 s2023-02-03 13:06:10,589 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/prcptot/QS-DEC/ssp245/simulations_30yAvg/prcptot_sea_BCCAQ2v2+ANUSPLIN300_KIOST-ESM_historical+ssp245_1950-2100_30ymean.nc.
[##                                      ] | 6% Completed | 31.02 s2023-02-03 13:06:11,154 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tx_mean/QS-DEC/ssp245/simulations_30yAvg/tx_mean_sea_BCCAQ2v2+ANUSPLIN300_CanESM5_historical+ssp245_1950-2100_30ymean.nc.
[##                                      ] | 6% Completed | 31.45 s2023-02-03 13:06:11,558 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tn_mean/QS-DEC/ssp585/simulations_30yAvg/tn_mean_sea_BCCAQ2v2+ANUSPLIN300_KIOST-ESM_historical+ssp585_1950-2100_30ymean.nc.
[##               

[##                                      ] | 6% Completed | 40.51 s2023-02-03 13:06:20,599 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/prcptot/QS-DEC/ssp126/simulations_30yAvg/prcptot_sea_BCCAQ2v2+ANUSPLIN300_MPI-ESM1-2-HR_historical+ssp126_1950-2100_30ymean.nc.
[##                                      ] | 6% Completed | 40.81 s2023-02-03 13:06:20,903 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tn_mean/QS-DEC/ssp585/simulations_30yAvg/tn_mean_sea_BCCAQ2v2+ANUSPLIN300_EC-Earth3_historical+ssp585_1950-2100_30ymean.nc.
[##                                      ] | 6% Completed | 41.36 s2023-02-03 13:06:21,501 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tx_mean/QS-DEC/ssp585/simulations_30yAvg/tx_mean_sea_BCCAQ2v2+ANUSPLIN300_BCC-CSM2-MR_historical+ssp585_1950-2100_30ymean.nc.
[##       

[##                                      ] | 6% Completed | 51.09 s2023-02-03 13:06:31,182 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tn_mean/QS-DEC/ssp126/simulations_30yAvg/tn_mean_sea_BCCAQ2v2+ANUSPLIN300_NorESM2-MM_historical+ssp126_1950-2100_30ymean.nc.
[##                                      ] | 6% Completed | 51.40 s2023-02-03 13:06:31,493 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/prcptot/QS-DEC/ssp126/simulations_30yAvg/prcptot_sea_BCCAQ2v2+ANUSPLIN300_UKESM1-0-LL_historical+ssp126_1950-2100_30ymean.nc.
[##                                      ] | 6% Completed | 51.87 s2023-02-03 13:06:31,962 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tx_mean/QS-DEC/ssp585/simulations_30yAvg/tx_mean_sea_BCCAQ2v2+ANUSPLIN300_CNRM-CM6-1_historical+ssp585_1950-2100_30ymean.nc.
[##         

[##                                      ] | 6% Completed | 62.58 s2023-02-03 13:06:42,682 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tx_mean/QS-DEC/ssp585/simulations_30yAvg/tx_mean_sea_BCCAQ2v2+ANUSPLIN300_MRI-ESM2-0_historical+ssp585_1950-2100_30ymean.nc.
[##                                      ] | 6% Completed | 62.91 s2023-02-03 13:06:43,000 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/prcptot/QS-DEC/ssp126/simulations_30yAvg/prcptot_sea_BCCAQ2v2+ANUSPLIN300_GFDL-ESM4_historical+ssp126_1950-2100_30ymean.nc.
[##                                      ] | 6% Completed | 63.27 s2023-02-03 13:06:43,360 - xscen.catalog - INFO - Parsing attributes with netCDF4 from /tank/jlavoie/ccdp/output/finalPCIC_corrige_QS-DEC/tn_mean/QS-DEC/ssp126/simulations_30yAvg/tn_mean_sea_BCCAQ2v2+ANUSPLIN300_MPI-ESM1-2-LR_historical+ssp126_1950-2100_30ymean.nc.
[##        

...

[####################################### ] | 98% Completed | 37m 1s2023-02-03 13:42:41,892 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/MS/ssp126/simulations_30yAvg/nr_cdd_mon_BCCAQ2v2+ANUSPLIN300_CNRM-CM6-1_historical+ssp126_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 2s2023-02-03 13:42:42,132 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/MS/ssp126/simulations_30yAvg/nr_cdd_mon_BCCAQ2v2+ANUSPLIN300_INM-CM4-8_historical+ssp126_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 2s2023-02-03 13:42:42,323 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/MS/ssp126/simulations_30yAvg/nr_cdd_mon_BCCAQ2v2+ANUSPLIN300_NorESM2-LM_historical+ssp126_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 2s2023-02-03 13:42:42,599 - xscen.catalog 

[####################################### ] | 98% Completed | 37m 16s2023-02-03 13:42:56,404 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/MS/ssp245/simulations_30yAvg/nr_cdd_mon_BCCAQ2v2+ANUSPLIN300_NorESM2-LM_historical+ssp245_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 16s2023-02-03 13:42:56,655 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/MS/ssp245/simulations_30yAvg/nr_cdd_mon_BCCAQ2v2+ANUSPLIN300_ACCESS-ESM1-5_historical+ssp245_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 24s2023-02-03 13:43:04,721 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/YS/ssp245/simulations_30yAvg/nr_cdd_ann_BCCAQ2v2+ANUSPLIN300_HadGEM3-GC31-LL_historical+ssp245_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 24s2023-02-03 13:43:04,939 - x

[####################################### ] | 98% Completed | 37m 30s2023-02-03 13:43:10,261 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/YS/ssp585/simulations_30yAvg/nr_cdd_ann_BCCAQ2v2+ANUSPLIN300_CanESM5_historical+ssp585_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 30s2023-02-03 13:43:10,537 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/YS/ssp585/simulations_30yAvg/nr_cdd_ann_BCCAQ2v2+ANUSPLIN300_HadGEM3-GC31-LL_historical+ssp585_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 30s2023-02-03 13:43:10,744 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/YS/ssp585/simulations_30yAvg/nr_cdd_ann_BCCAQ2v2+ANUSPLIN300_MIROC6_historical+ssp585_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 30s2023-02-03 13:43:10,881 - xscen.catal

[####################################### ] | 98% Completed | 37m 51s2023-02-03 13:43:31,256 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/YS/ssp126/simulations_30yAvg/nr_cdd_ann_BCCAQ2v2+ANUSPLIN300_CMCC-ESM2_historical+ssp126_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 51s2023-02-03 13:43:31,449 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/YS/ssp126/simulations_30yAvg/nr_cdd_ann_BCCAQ2v2+ANUSPLIN300_MPI-ESM1-2-LR_historical+ssp126_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 51s2023-02-03 13:43:31,569 - xscen.catalog - INFO - Parsing attributes from Zarr /tank/jlavoie/ccdp/output/final/nr_cdd/YS/ssp126/simulations_30yAvg/nr_cdd_ann_BCCAQ2v2+ANUSPLIN300_ACCESS-ESM1-5_historical+ssp126_1950-2100_30ymean.zarr.
[####################################### ] | 98% Completed | 37m 51s2023-02-03 13:43:31,674 - xsce

,id,type,processing_level,bias_adjust_institution,bias_adjust_project,mip_era,activity,driving_institution,driving_model,institution,source,experiment,member,xrfreq,frequency,variable,domain,date_start,date_end,version,format,path
28602,climatedata_CMIP6_ScenarioMIP_NorESM2-MM_ssp12...,simulation,30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,NorESM2-MM,ssp126,None,MS,mon,"(r20mm, r20mm_delta_1971_2000, r20mm_delta_199...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,zarr,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp12...
29028,climatedata_CMIP6_ScenarioMIP_MIROC6_ssp126_CA...,simulation,30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,MIROC6,ssp126,None,MS,mon,"(r20mm, r20mm_delta_1971_2000, r20mm_delta_199...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,zarr,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp12...
29454,climatedata_CMIP6_ScenarioMIP_INM-CM5-0_ssp126...,simulation,30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,INM-CM5-0,ssp126,None,MS,mon,"(r20mm, r20mm_delta_1971_2000, r20mm_delta_199...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,zarr,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp12...
29880,climatedata_CMIP6_ScenarioMIP_CNRM-ESM2-1_ssp1...,simulation,30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,CNRM-ESM2-1,ssp126,None,MS,mon,"(r20mm, r20mm_delta_1971_2000, r20mm_delta_199...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,zarr,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp12...
30306,climatedata_CMIP6_ScenarioMIP_BCC-CSM2-MR_ssp1...,simulation,30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,BCC-CSM2-MR,ssp126,None,MS,mon,"(r20mm, r20mm_delta_1971_2000, r20mm_delta_199...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,zarr,/tank/jlavoie/ccdp/output/final/r20mm/MS/ssp12...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10247544,climatedata_CMIP6_ScenarioMIP_HadGEM3-GC31-LL_...,simulation,30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,HadGEM3-GC31-LL,ssp585,None,QS-DEC,qtr,"(prcptot, prcptot_delta_1971_2000, prcptot_del...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...
10247545,climatedata_CMIP6_ScenarioMIP_EC-Earth3_ssp585...,simulation,30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,EC-Earth3,ssp585,None,QS-DEC,qtr,"(prcptot, prcptot_delta_1971_2000, prcptot_del...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...
10247546,climatedata_CMIP6_ScenarioMIP_CNRM-CM6-1_ssp58...,simulation,30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,CNRM-CM6-1,ssp585,None,QS-DEC,qtr,"(prcptot, prcptot_delta_1971_2000, prcptot_del...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...
10247547,climatedata_CMIP6_ScenarioMIP_INM-CM4-8_ssp585...,simulation,30ymean,PCIC,climatedata,CMIP6,ScenarioMIP,None,None,None,INM-CM4-8,ssp585,None,QS-DEC,qtr,"(prcptot, prcptot_delta_1971_2000, prcptot_del...",CAN-PCIC,1950-01-01 00:00,2100-12-31 23:00,None,nc,/tank/jlavoie/ccdp/output/finalPCIC_corrige_QS...


# make ensemble-per-delta for CDCA

In [4]:
# make ensemble-per-delta
for exp_pcic in ['ssp245','ssp585']:
    for var in ['prcptot']:
        for xrfreq in ['AS-JAN', 'QS-DEC']:
            if not pcat.exists_in_cat(experiment=exp_pcic, xrfreq=xrfreq,processing_level ='ensemble-per-delta'):
                datasets_pcic={}

                #get all delta members of ensemble. replace absolute by percentage for prcptot.
                #for p in pcic_paths[xrfreq][var]: # both line should do the same
                for p in pcat.search(
                    variable=var, xrfreq=xrfreq, processing_level ='30ymean', experiment=exp_pcic).df.path:
                    ds = xr.open_dataset(p, decode_timedelta=False)
                    if var == 'prcptot': # put delta in % for pr
                        ln= ds[f'{var}_delta_1991_2020'].attrs['long_name'].replace('delta', 'percentage delta')
                        ds[f'{var}_delta_1991_2020'] = ds[f'{var}_delta_1991_2020']/ds[var] *100
                        ds[f'{var}_delta_1991_2020'].attrs['units']='%'
                        ds[f'{var}_delta_1991_2020'].attrs['long_name']=ln
                    datasets_pcic[p]=ds

                # ensemble stats
                ens_pcic = xs.ensembles.ensemble_stats(
                    datasets=datasets_pcic,
                    statistics={"ensemble_percentiles": {'split': True}},
                    common_attrs_only=False,
                    create_kwargs=dict(chunks={'horizon':-1}),
                    to_level ='ensemble-per-delta'
                )

                # add all attrs to make pcic data fit xscen catalog format
                ens_pcic= ens_pcic[[f'{var}_delta_1991_2020_{p}' for p in['p10','p50','p90']]].chunk({'lat':-1, 'lon':-1})
                ens_pcic.attrs['cat:xrfreq']=xrfreq
                ens_pcic.attrs['cat:variable']=var
                ens_pcic.attrs['cat:type']='simulation'
                ens_pcic.attrs['cat:bias_adjust_institution']= 'PCIC'
                ens_pcic.attrs['cat:bias_adjust_project']='climatedata'
                ens_pcic.attrs['cat:mip_era']= 'CMIP6'
                ens_pcic.attrs['cat:activity']= 'ScenarioMIP'
                ens_pcic.attrs['cat:experiment']= exp_pcic
                ens_pcic.attrs['cat:domain']='CAN-PCIC'
                ens_pcic.attrs['cat:id']=xs.catalog.generate_id(ens_pcic).iloc[0]

                # save it
                with ProgressBar():
                    path = CONFIG['paths']['ensembles'].format(var=var, **xs.utils.get_cat_attrs(ens_pcic))
                    xs.save_to_zarr(ens_pcic, path, mode ='o')
                    pcat.update_from_ds(ens_pcic, path)


INFO:xscen.catalog:An entry exists for: {'experiment': 'ssp245', 'xrfreq': 'AS-JAN', 'processing_level': 'ensemble-per-delta'}
INFO:xscen.catalog:An entry exists for: {'experiment': 'ssp245', 'xrfreq': 'QS-DEC', 'processing_level': 'ensemble-per-delta'}
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'netcdf4' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'h5netcdf' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'scipy' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'netcdf4' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'h5netcdf' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends

/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'netcdf4' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'h5netcdf' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'scipy' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'netcdf4' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'h5netcdf' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'scipy' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/backends/plugins.py:139: RuntimeWarning: 'netcdf4' fails while guessing
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/x

[########################################] | 100% Completed | 737.03 ms


INFO:xscen.catalog:File format not specified. Adding it as 'zarr' based on file name.
INFO:xscen.ensembles:Creating ensemble with 26 simulations and calculating ensemble_percentiles.
/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xclim/core/utils.py:462: RuntimeWarning: All-NaN slice encountered


[########################################] | 100% Completed | 104.96 ms
[########################################] | 100% Completed | 102.00 ms


/exec/jlavoie/.conda/espo-g/lib/python3.9/site-packages/xarray/conventions.py:232: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.


[########################################] | 100% Completed | 2.74 ss


INFO:xscen.catalog:File format not specified. Adding it as 'zarr' based on file name.


# PC ensemble ( regridded on CDCA grid)

In [3]:
# regrid pc onto cd grid

for exp_espo in ['ssp245','ssp370']:
    for  espo_level in ['ensemble-climatology', 'ensemble-abs-delta-1991-2020', 'ensemble-per-delta-1991-2020']:
        for var in ['tn_mean', 'tx_mean','prcptot', 'tg_mean']:
            for xrfreq in ['QS-DEC','AS-JAN']:
                #only per-delta with prcptot
                if not  (espo_level=='ensemble-per-delta-1991-2020' and var in ['tn_mean', 'tx_mean', 'tg_mean']):
                
                    delta = '_delta_1991_2020' if 'delta' in espo_level else ""
                    variable= [f'{var}{delta}_{p}' for p in ['p10','p50','p90']]

                    if not pcat.exists_in_cat(bias_adjust_project='ESPO-G6-R2',domain='CAN-PCIC',
                                         variable=variable[1],xrfreq=xrfreq, experiment=exp_espo,
                                              processing_level=espo_level):

                        # get pc ensemple
                        ens_espo= pcat_espo.search(processing_level=espo_level,
                                                   experiment=exp_espo,
                                                   variable =variable,
                                                 xrfreq=xrfreq, domain='NAM-rdrs' ).to_dask(
                            xarray_open_kwargs={'decode_timedelta': False, 'chunks':{'horizon':-1, 'time':-1}})
                        ens_espo = ens_espo.chunk({'rlat':-1, 'rlon':-1, 'time':1})


                        # regrid onto pcic grid
                        #get pcic grid
                        ens_pcic= pcat.search(xrfreq=xrfreq, variable=variable[1], experiment='ssp245',
                                              bias_adjust_project='climatedata',processing_level='ensemble-30ymean',
                                              domain='CAN-PCIC').to_dask(**tdd).isel(time=0).drop_vars('time')

                        #create mask
                        mask_args = {
                            "variable": variable[1],
                            "where_operator": "<",
                            "where_threshold": 100000000,
                            "mask_nans": True
                        }
                        ens_espo["mask"] = xs.create_mask(ens_espo, mask_args=mask_args)
                        ens_pcic["mask"] = xs.create_mask(ens_pcic, mask_args=mask_args)

                        #regrid
                        ens_espo = xs.regrid_dataset(
                                        ds=ens_espo,
                                        weights_location=CONFIG['paths']['weights_location'],
                                        ds_grid=ens_pcic,
                                        regridder_kwargs={"extrap_method": "inverse_dist"},
                                        to_level= espo_level
                                    )
                        ens_espo.attrs['cat:bias_adjust_project']='ESPO-G6-R2'
                        ens_espo.attrs['cat:id']=xs.catalog.generate_id(ens_espo).iloc[0]
                        ens_espo['horizon']=ens_espo.horizon.astype(str)
                        display(ens_espo)

                        # save it
                        with ProgressBar():
                            path = CONFIG['paths']['ensembles'].format(var=var, **xs.utils.get_cat_attrs(ens_espo))
                            xs.save_to_zarr(ens_espo, path, mode ='o')
                            pcat.update_from_ds(ens_espo, path)


INFO:xscen.catalog:An entry exists for: {'bias_adjust_project': 'ESPO-G6-R2', 'domain': 'CAN-PCIC', 'variable': 'tn_mean_p50', 'xrfreq': 'QS-DEC', 'experiment': 'ssp245', 'processing_level': 'ensemble-climatology'}
INFO:xscen.catalog:An entry exists for: {'bias_adjust_project': 'ESPO-G6-R2', 'domain': 'CAN-PCIC', 'variable': 'tn_mean_p50', 'xrfreq': 'AS-JAN', 'experiment': 'ssp245', 'processing_level': 'ensemble-climatology'}
INFO:xscen.catalog:An entry exists for: {'bias_adjust_project': 'ESPO-G6-R2', 'domain': 'CAN-PCIC', 'variable': 'tx_mean_p50', 'xrfreq': 'QS-DEC', 'experiment': 'ssp245', 'processing_level': 'ensemble-climatology'}
INFO:xscen.catalog:An entry exists for: {'bias_adjust_project': 'ESPO-G6-R2', 'domain': 'CAN-PCIC', 'variable': 'tx_mean_p50', 'xrfreq': 'AS-JAN', 'experiment': 'ssp245', 'processing_level': 'ensemble-climatology'}
INFO:xscen.catalog:An entry exists for: {'bias_adjust_project': 'ESPO-G6-R2', 'domain': 'CAN-PCIC', 'variable': 'prcptot_p50', 'xrfreq': 'QS

<xarray.Dataset>
Dimensions:                      (time: 14, lat: 510, lon: 1068)
Coordinates:
    horizon                      <U4 dask.array<chunksize=(), meta=np.ndarray>
    rotated_pole                 float32 ...
  * time                         (time) datetime64[ns] 1989-01-01 ... 2071-01-01
  * lat                          (lat) float64 41.04 41.12 41.21 ... 83.37 83.46
  * lon                          (lon) float64 -141.0 -140.9 ... -52.13 -52.04
Data variables:
    prcptot_delta_1991_2020_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 510, 1068), meta=np.ndarray>
    prcptot_delta_1991_2020_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 510, 1068), meta=np.ndarray>
    prcptot_delta_1991_2020_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 510, 1068), meta=np.ndarray>
Attributes: (12/43)
    Notes:                        Regridded on the grid of RDRS v2.1, then bi...
    activity_id:                  CMIP
    branch_time_in_child:         0.0
    cat:_data_format_:            zarr
    cat:activity:                 ScenarioMIP
    cat:bias_adjust_institution:  Ouranos
    ...                           ...
    version:                      1.1.0
    cat:date_start:               1989-01-01 00:00
    cat:date_end:                 2071-01-01 00:00
    cat:path:                     /jarre/scenario/jlavoie/ESPO-G6/ensemble/en...
    intake_esm_dataset_key:       ESPO-G6_CMIP6_ScenarioMIP_ssp370_NAM-rdrs.N...
    history:                      [2023-08-01 09:41:40] regridded with argume...

Removing /jarre/scenario/jlavoie/plat-comp/ensembles/CAN-PCIC/ESPO-G6-R2_CMIP6_ScenarioMIP_ssp370_CAN-PCIC/ESPO-G6-R2_CMIP6_ScenarioMIP_ssp370_CAN-PCIC_CAN-PCIC_AS-JAN_prcptot_ensemble-abs-delta-1991-2020.zarr/prcptot_delta_1991_2020_p90 to overwrite.
Removing /jarre/scenario/jlavoie/plat-comp/ensembles/CAN-PCIC/ESPO-G6-R2_CMIP6_ScenarioMIP_ssp370_CAN-PCIC/ESPO-G6-R2_CMIP6_ScenarioMIP_ssp370_CAN-PCIC_CAN-PCIC_AS-JAN_prcptot_ensemble-abs-delta-1991-2020.zarr/prcptot_delta_1991_2020_p50 to overwrite.
Removing /jarre/scenario/jlavoie/plat-comp/ensembles/CAN-PCIC/ESPO-G6-R2_CMIP6_ScenarioMIP_ssp370_CAN-PCIC/ESPO-G6-R2_CMIP6_ScenarioMIP_ssp370_CAN-PCIC_CAN-PCIC_AS-JAN_prcptot_ensemble-abs-delta-1991-2020.zarr/prcptot_delta_1991_2020_p10 to overwrite.
[########################################] | 100% Completed | 9.66 ss


INFO:xscen.catalog:File format not specified. Adding it as 'zarr' based on file name.
INFO:xscen.catalog:An entry exists for: {'bias_adjust_project': 'ESPO-G6-R2', 'domain': 'CAN-PCIC', 'variable': 'tg_mean_delta_1991_2020_p50', 'xrfreq': 'QS-DEC', 'experiment': 'ssp370', 'processing_level': 'ensemble-abs-delta-1991-2020'}
INFO:xscen.catalog:An entry exists for: {'bias_adjust_project': 'ESPO-G6-R2', 'domain': 'CAN-PCIC', 'variable': 'tg_mean_delta_1991_2020_p50', 'xrfreq': 'AS-JAN', 'experiment': 'ssp370', 'processing_level': 'ensemble-abs-delta-1991-2020'}
INFO:xscen.catalog:An entry exists for: {'bias_adjust_project': 'ESPO-G6-R2', 'domain': 'CAN-PCIC', 'variable': 'prcptot_delta_1991_2020_p50', 'xrfreq': 'QS-DEC', 'experiment': 'ssp370', 'processing_level': 'ensemble-per-delta-1991-2020'}
INFO:xscen.catalog:An entry exists for: {'bias_adjust_project': 'ESPO-G6-R2', 'domain': 'CAN-PCIC', 'variable': 'prcptot_delta_1991_2020_p50', 'xrfreq': 'AS-JAN', 'experiment': 'ssp370', 'processin